# Classificação de tumores de mama. 

<p style="text-align: justify">O objetivo da análise foi criar um modelo de machine learning que possa classificar tumores de câncer de mama (benignos ou malignos) atráves de um <em>dataset</em> fornecido pelo <a href="https://archive.ics.uci.edu/ml/datasets.php">Repositório de Machine Learning da UCI</a>. Os modelos preditivos gerados podem ser extremamente úteis para diagnóstico da doença e consequentemente ajudar na saúde pública.</p>

### Descrição dos dados:

<p style="text-align: justify">Os dados foram coletados a partir de imagens computadorizadas após um aspirado por agulha da massa da mama. Foram extraídos 32 atributos para criação do <em>dataset</em>. Mais informações a respeito do <em>dataset</em> podem ser adquiridas <a href="https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29"> aqui</a>:</p>

####  Criadores 

1. Dr. William H. Wolberg, General Surgery Dept.
University of Wisconsin, Clinical Sciences Center
Madison, WI 53792
wolberg '@' eagle.surgery.wisc.edu

2. W. Nick Street, Computer Sciences Dept.
University of Wisconsin, 1210 West Dayton St., Madison, WI 53706
street '@' cs.wisc.edu 608-262-6619

3. Olvi L. Mangasarian, Computer Sciences Dept.
University of Wisconsin, 1210 West Dayton St., Madison, WI 53706
olvi '@' cs.wisc.edu

#### Referência

Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

### Importando bibliotecas

In [78]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import RandomizedSearchCV

### Importando os dados

In [79]:
#Importando os dados para um Dataframe
dados = pd.read_csv('data.csv')
#Visualizando os dados
dados.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


### Explorando os dados

In [80]:
#Verificando o tamanho do dataset(linhas, colunas)
dados.drop(['Unnamed: 32'], axis=1, inplace=True)
dados.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [81]:
#Verificando a existência de dados nulos no dataset
pd.DataFrame(dados.isnull().sum(), columns=['Dados_nulos'])

,Dados_nulos
id,0
diagnosis,0
radius_mean,0
texture_mean,0
perimeter_mean,0
area_mean,0
smoothness_mean,0
compactness_mean,0
concavity_mean,0
concave points_mean,0


In [82]:
#Verificando estatísticas descritivas do dataset
dados.describe()

,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


In [83]:
#Verificando o balanceamento dos dados que serão previsto
dados['diagnosis'].value_counts()

B    357
M    212
Name: diagnosis, dtype: int64

<p style="text-align: justify">Há um desbalanceamento entre as classes dos dados o que pode diminuir o poder de previsão do modelo em prever os dados que são mais raros (tumores malignos(M)). Nesse caso, é esperado que o modelo consiga prever melhor os dados da categoria benigno(B).</p> 

### Criando o modelo

### Construindo o modelo

<p>Separando os eixos X e y do modelo.</p>

In [84]:
#Separando os dados da variável independente do modelo
X = dados.drop(['diagnosis','id'], axis=1)
#Separando os dados da variável dependente do modelo
y = dados['diagnosis']

### Normalização das variáveis

Isso foi feito para garantir que uma variável não infuencie exageradamente o modelo simplesmente pela sua escala de medição original.

In [85]:
#Padronizando as escalas das variáveis do modelo                
X=StandardScaler().fit_transform(X)

In [86]:
#Transformando os valores da coluna em 0(Benigno) e 1(Maligno)
y=LabelEncoder().fit_transform(y)

### Separando dados de treino e teste

In [87]:
#Separando os dados de treino e de teste                
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3, random_state=42)

### SVM - (<em>Support Vector Machine</em>)

O <em>Support Vector Machine</em> é um modelo de machine learning capaz de realizar tanto classifcações lineares como não lineares. O modelo foi gerado usando 3 parâmetros (C, gamma e kernel) e utilizando valores de maneira arbitrária, para que o modelo gerado pudesse servir de base para otimização de parâmetros com validação cruzada (*cross validation*). 

In [88]:
#Instanciando o modelo
model=SVC(C=10, gamma=0.0001, kernel="rbf")

In [89]:
#Treinando o modelo
model.fit(X_train,y_train)

SVC(C=10, gamma=0.0001)

In [90]:
#Testando a predição do modelo com os dados de teste
model_pred = model.predict(X_test)

### Avaliando a métricas do modelo

In [91]:
#Visualizando a matriz de confusão
print("Matriz de confusão:\n",confusion_matrix(y_test,model_pred).round(2))
#Visualizando métricas de classificação
print("Métricas de classificação:\n",classification_report(y_test,model_pred))
#Visualizando acurácia do modelo
print("Acurácia:\n",accuracy_score(y_test,model_pred).round(2))
#Visualizando o AUC
print("AUC:\n",roc_auc_score(y_test,model_pred).round(2))

Matriz de confusão:
 [[108   0]
 [  7  56]]
Métricas de classificação:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97       108
           1       1.00      0.89      0.94        63

    accuracy                           0.96       171
   macro avg       0.97      0.94      0.95       171
weighted avg       0.96      0.96      0.96       171

Acurácia:
 0.96
AUC:
 0.94


<p style="text-align: justify">O modelo teve um total de 96% de acurária, ou seja, o modelo gerado tem a capacidade de detectar corretamente as duas classes de tumores (Beningnos e Malignos) corretamente a partir das variáveis do <em>dataset</em> em 96 em 100 vezes. Entretanto, quando avaliamos a revocação do modelo, que mede a força do modelo em prever um resultado positivo (tumores malignos), o modelo teve um poder de previsão de 89%. Além disso, quando avaliamos a especificidade, que mede a capacidade do modelo em prever o resultado negativo (tumores benignos), o modelo teve um poder de previsão de 100%. Provavelmente, o modelo conseguiu prever melhor a classe com maior número (benignos).</p>

<p style="text-align: justify">Adicionalmente, uma métrica capaz de avaliar o <em>trade-off</em> entre as métricas sensibilidade e especificidade é AUC (<em>Area under curve</em>). A partir dos valores de AUC foi possível verificar que o modelo teve 94% de eficácia. Assim, apesar do modelo ter uma boa acurácia geral para detecção de tumores (benignos e malignos), vou tentar melhorar esses valores otimizando os parâmetros do modelo.</p>

### Otimizando o modelo

<p style="text-align: justify">A partir dos parâmetros escolhidos anteriormente, será realizado uma busca para tentar encontrar valores dos parâmetros que possam melhorar as métricas de avaliação do modelo. Além disso, vou utilizar um método de validação cruzada que leva em consideração o desbalanceamento das categorias (<em>Stratified Shuffle Split</em>). Abaixo segue os resultados da busca de parâmetros:</p>

In [92]:
split = StratifiedShuffleSplit(n_splits =10, test_size=0.3, random_state=42)

seed=301
np.random.seed(seed)

parametros = {
    "C":[0.1,1, 10, 100, 1000],
    "gamma":[1,0.1,0.01,0.001,0.0001],
    "kernel":["rbf","linear","poly"]
}

model_1 = RandomizedSearchCV(SVC(),
                    parametros,n_iter = 75, 
                    cv=split, random_state = seed)

model_1.fit(X,y)
resultados = pd.DataFrame(model_1.cv_results_)
resultados.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_gamma,param_C,params,split0_test_score,split1_test_score,...,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.012492,0.001499,0.004098,0.000700,rbf,1,0.1,"{'kernel': 'rbf', 'gamma': 1, 'C': 0.1}",0.625731,0.625731,...,0.625731,0.625731,0.625731,0.625731,0.625731,0.625731,0.625731,0.625731,0.000000,67
1,0.002599,0.000663,0.000700,0.000458,linear,1,0.1,"{'kernel': 'linear', 'gamma': 1, 'C': 0.1}",0.982456,0.988304,...,0.976608,0.976608,0.970760,0.982456,0.964912,0.982456,0.988304,0.977778,0.008187,1
2,0.003299,0.000900,0.000699,0.000643,poly,1,0.1,"{'kernel': 'poly', 'gamma': 1, 'C': 0.1}",0.941520,0.959064,...,0.941520,0.959064,0.959064,0.953216,0.947368,0.964912,0.935673,0.950877,0.009135,40
3,0.006798,0.000401,0.002794,0.000595,rbf,0.1,0.1,"{'kernel': 'rbf', 'gamma': 0.1, 'C': 0.1}",0.935673,0.929825,...,0.929825,0.900585,0.941520,0.953216,0.900585,0.970760,0.941520,0.935088,0.020667,52
4,0.002098,0.000538,0.000700,0.000458,linear,0.1,0.1,"{'kernel': 'linear', 'gamma': 0.1, 'C': 0.1}",0.982456,0.988304,...,0.976608,0.976608,0.970760,0.982456,0.964912,0.982456,0.988304,0.977778,0.008187,1


### Melhor cobinação de parâmetros para o modelo 

In [93]:
best = model_1.best_estimator_
print(best)

SVC(C=0.1, gamma=1, kernel='linear')


A partir da análise acima foi possível encontrar valores dos parâmetros que melhoram o poder de classificação do modelo. Dessa forma, será verificado se o modelo possui melhores atributos do que o primeiro.

## Avaliando o modelo com parâmetros otimizados

In [94]:
model_otimizado=SVC(C=0.1, gamma=1, kernel='linear')
model_otimizado.fit(X_train,y_train)
model_otimizado_pred = model_otimizado.predict(X_test)

In [95]:
#Visualizando a matriz de confusão
print("Matriz de confusão:\n",confusion_matrix(y_test,model_otimizado_pred))
#Visualizando métricas de classificação
print("Métricas de classificação:\n",classification_report(y_test,model_otimizado_pred))
#Visualizando acurácia do modelo
print("Acurácia:\n",accuracy_score(y_test,model_otimizado_pred))
#Visualizando o AUC
print("AUC:\n",roc_auc_score(y_test,model_otimizado_pred))

Matriz de confusão:
 [[107   1]
 [  2  61]]
Métricas de classificação:
               precision    recall  f1-score   support

           0       0.98      0.99      0.99       108
           1       0.98      0.97      0.98        63

    accuracy                           0.98       171
   macro avg       0.98      0.98      0.98       171
weighted avg       0.98      0.98      0.98       171

Acurácia:
 0.9824561403508771
AUC:
 0.9794973544973545


<p style="text-align: justify">Como pode ser visto acima, o modelo otimizado conseguiu melhorar as métricas mais importante e consequentemente aumentar o poder de previsão. Com relação a métrica acurácia, o modelo otimizado conseguiu 98% contra 96% do modelo não otimizado. Já para a métrica de sensibilidade, ou seja a capacidade do modelo detectar tumores malignos o modelo otimizado conseguiu 97% contra 89% do modelo não otimizado, o que foi de grande valia. Apesar da métrica especificidade no modelo otimizado diminuir (de 100% para 99%), isso era esperado uma vez que a medida que um modelo acerta mais uma das classes, o mesmo tende a errar para a outra (<em>trade-off</em>). Além disso, houve também aumento da métrica de AUC que teve um desempenho 97% do modelo otimizado contra 94% do modelo não otimizado</p> 
<p style="text-align: justify">Dessa maneira, a partir da otimização de parâmetros e validação cruzada foi possível conseguir um modelo com melhores métricas na avalição. Esse tipo de análise pode ser de grande valia, pois pode ajudar no diagnóstico de câncer de mama a partir de variáveis, como coletadas deste <em>dataset</em>, e possibilitar avanços no combate da doença.</p>   